<a href="https://colab.research.google.com/github/SHASHANK8412/Restaurant-Ratings/blob/main/Restaraunt_Recomendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries




In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


Loading the dataset

In [20]:
df=pd.read_csv('Dataset .csv')
print(df.isna().sum())
print(df.head())

# Force object type for text columns
text_cols = ['Restaurant Name', 'City', 'Cuisines']
for col in text_cols:
    df[col] = df[col].astype(str)


Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64
   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Manda

Preprocessing

In [11]:
for col in df.columns:
  if df[col].dtype=='object':
    df[col]=df[col].fillna(df[col].mode()[0])
  else:
    df[col]=df[col].fillna(df[col].mean())


le=LabelEncoder()
for col in df.select_dtypes(include=['object']).columns:
  df[col]=le.fit_transform(df[col])

In [12]:
df['Norm Price'] = (df['Average Cost for two'] - df['Average Cost for two'].min()) / \
                   (df['Average Cost for two'].max() - df['Average Cost for two'].min())
df['Norm Rating'] = df['Aggregate rating'] / df['Aggregate rating'].max()


In [18]:

# Ensure all cuisines are strings
df['Cuisines'] = df['Cuisines'].astype(str)

# TF-IDF Vectorization on Cuisines
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Cuisines'])

# Combine text + numeric features
features = np.hstack((tfidf_matrix.toarray(),
                      df[['Norm Price', 'Norm Rating']].values))


In [22]:
def recommend_restaurants(user_cuisine, max_budget, min_rating, top_n=5):
    """
    Recommend restaurants based on cuisine, budget, and rating
    """
    # Filter dataset by budget and rating
    filtered_df = df[(df['Average Cost for two'] <= max_budget) &
                     (df['Aggregate rating'] >= min_rating)]

    if filtered_df.empty:
        return "No restaurants found matching your criteria."


    user_vec = tfidf.transform([user_cuisine]).toarray()
    user_features = np.hstack((user_vec,
                               [[max_budget/df['Average Cost for two'].max(),
                                 min_rating/df['Aggregate rating'].max()]]))

    # Compute similarity with all restaurants
    sims = cosine_similarity(user_features, features).flatten()

    # Get top restaurants
    top_indices = sims.argsort()[-top_n:][::-1]

    return df.iloc[top_indices][['Restaurant Name', 'City', 'Cuisines',
                                  'Average Cost for two', 'Aggregate rating']]


if __name__ == "__main__":
    print("---- Restaurant Recommendation System ----")
    cuisine = input("Enter preferred cuisine: ")
    budget = int(input("Enter maximum budget for two people: "))
    rating = float(input("Enter minimum rating (e.g. 3.5): "))

    recommendations = recommend_restaurants(cuisine, budget, rating, top_n=5)
    print("\nTop Restaurant Recommendations:\n")
    print(recommendations)

---- Restaurant Recommendation System ----
Enter preferred cuisine: Italian
Enter maximum budget for two people: 2000
Enter minimum rating (e.g. 3.5): 3

Top Restaurant Recommendations:

                    Restaurant Name         City  \
6659       The California Boulevard    New Delhi   
21    The Food Hall by Todd English  Taguig City   
8608          Kenny Rogers Roasters        Noida   
3907               Big Fat Sandwich    New Delhi   
2626               Big Fat Sandwich    New Delhi   

                                Cuisines  Average Cost for two  \
6659  American, Asian, European, Seafood                  2000   
21     American, Asian, Italian, Seafood                  1800   
8608                            American                  1300   
3907                            American                   800   
2626                            American                   800   

      Aggregate rating  
6659               4.6  
21                 4.5  
8608               3.7  
390